# Import

In [ ]:
import sys
# 캐시된 모듈 제거
for mod in list(sys.modules.keys()):
    if 'NPR' in mod or 'npr' in mod or 'networks' in mod:
        del sys.modules[mod]

# 이제 정상적으로 import
from model.NPR.npr_model import NPR

In [ ]:
import sys
for mod in list(sys.modules.keys()):
    if any(x in mod for x in ['NPR', 'npr', 'networks', 'sgs', 'method']):
        del sys.modules[mod]

In [ ]:
import os
from pathlib import Path 
from typing import Optional, Literal
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

from utils.data.dataset import CorruptedDataset
from utils.visual.visualizer import DatasetVisualizer
from utils.eval.metrics import PredictionCollector, MetricsCalculator

# SGS method import
from model.method.sgs import LGradSGS, SGSConfig
from model.LGrad.lgrad_model import LGrad

# GPU and Model select

In [ ]:
!nvidia-smi

In [ ]:
DEVICE="cuda:0"
MODEL_LIST = ["lgrad", "npr"]
MODEL = MODEL_LIST[0]

# Dataloader

In [ ]:
ROOT = "corrupted_dataset"
DATASETS = ["corrupted_test_data_biggan", "corrupted_test_data_crn", "corrupted_test_data_cyclegan", "corrupted_test_data_deepfake", "corrupted_test_data_gaugan", "corrupted_test_data_imle", "corrupted_test_data_progan", "corrupted_test_data_san", "corrupted_test_data_seeingdark", "corrupted_test_data_stargan", "corrupted_test_data_stylegan", "corrupted_test_data_stylegan2", "corrupted_test_data_whichfaceisreal"]
CORRUPTIONS = ["original", "contrast", "fog", "gaussian_noise", "jpeg_compression", "motion_blur", "pixelate"]
if MODEL == "lgrad":
    transform=transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )
else:
    transform=transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )

In [ ]:
dataset = CorruptedDataset(
    root= ROOT,
    datasets=DATASETS,
    corruptions=CORRUPTIONS,
    transform=transform
)

print(f"Total samples: {len(dataset)}")

# Model load

In [ ]:
from model.LGrad.lgrad_model import LGrad
from model.NPR.npr_model import NPR

#LGrad
STYLEGAN_WEIGHTS_ROOT="model/LGrad/weights/karras2019stylegan-bedrooms-256x256_discriminator.pth"
CLASSIFIER_WEIGHTS_ROOT="model/LGrad/weights/LGrad-Pretrained-Model/LGrad-4class-Trainon-Progan_car_cat_chair_horse.pth"

#NPR
NPR_WEIGHTS_ROOT="model/NPR/weights/NPR.pth"

if MODEL == "lgrad":
    model = LGrad(
        stylegan_weights=STYLEGAN_WEIGHTS_ROOT,
        classifier_weights=CLASSIFIER_WEIGHTS_ROOT,
        device=DEVICE
    )
    model_name="LGrad"
    LAMBDA=0.03
    DELTA=0.01
    ITERATIONS=5
    STEP_SIZE=0.1
elif MODEL == "npr":
    model = NPR(
        weights=NPR_WEIGHTS_ROOT,
        device=DEVICE
    )
    model_name="NPR"
    LAMBDA=0.005
    DELTA=0.01
    ITERATIONS=3
    STEP_SIZE=0.1


In [ ]:
from model.method.sgs import SGSConfig, UnifiedSGS
sgs_config = SGSConfig(
    K=4,
    model=model_name,
    denoise_target="artifact",
    huber_tv_lambda=LAMBDA,
    huber_tv_delta=DELTA,
    huber_tv_iterations=ITERATIONS,
    huber_tv_step_size=STEP_SIZE,

    device=DEVICE,
)

model = UnifiedSGS(model, sgs_config)

model.model.eval()

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

In [ ]:
from torch.utils.data import Subset, DataLoader
import random

# Evaluation
calc = MetricsCalculator()
all_results = {}

for dataset_name in DATASETS:
    for corruption in CORRUPTIONS:
        combination_indices = [
            i for i, s in enumerate(dataset.samples)
            if s['dataset'] == dataset_name and s['corruption'] ==corruption
        ]

        if len(combination_indices) == 0:
            print(f"{dataset_name}-{corruption}: 샘플 없음, 스킵")
            continue

        print(f"\n{'='*60}")
        print(f"평가 중: {dataset_name}-{corruption}")
        print(f"샘플 수: {len(combination_indices)}")
        print(f"{'='*60}")

        # Subset과 DataLoader 생성
        subset = Subset(dataset, combination_indices)
        dataloader = DataLoader(
            subset,
            batch_size=32,
            shuffle=False,
            num_workers=4,
            # collate_fn=collate_fn,
            drop_last=True
        )

        # 평가
        metrics = calc.evaluate(
            model=model,
            dataloader=dataloader,
            device=DEVICE,
            name=f"{dataset_name}-{corruption}"
        )

        # 즉시 결과 출력
        print(f"\n결과:")
        print(f"  Accuracy: {metrics['accuracy']*100:.2f}%")
        print(f"  AUC:      {metrics['auc']*100:.2f}%")
        print(f"  AP:       {metrics['ap']*100:.2f}%")
        print(f"  F1:       {metrics['f1']*100:.2f}%")

        # 결과 저장
        all_results[(dataset_name, corruption)] = metrics

# 전체 결과 테이블 출력
print(f"\n\n{'='*60}")
print("전체 결과 요약")
print(f"{'='*60}\n")
calc.print_results_table()
calc.summarize_by_corruption(all_results)
calc.summarize_by_dataset(all_results)